Script to quantify error induced by using TMY weather with actual year load in a variety of locations with a variety of utility rate structures

Requires:
nrel-pysam
requests
numpy
pandas

In [2]:
import numpy as np
import pandas as pd
import json
import os
import csv

import PySAM.Battery as battery_model
import PySAM.Pvsamv1 as pv_model
import PySAM.Utilityrate5 as utility_rate
import PySAM.Cashloan as cashloan
import PySAM.ResourceTools
import PySAM.UtilityRateTools

In [1]:
import sys
print(sys.executable)

/Users/bspeetle/Desktop/repo/SAM-analyses/2025/battwatts_sensitivity/venv/bin/python


In [3]:
def get_pysam_json(json_file_path):
    """
    Open a PySAM JSON file and return as a dictionary
    """
    with open(json_file_path) as f:
        dic = json.load(f)
    return dic


def get_load_profile(load_path, desired_timestep_minutes):
    """
    Get data out of a CSV file
    Original ResStock data is 15 minute kWh data - need hourly for TMY comparison and to convert to kW
    """
    df = pd.read_csv(load_path)
    timeseries = df["out.electricity.total.energy_consumption"].values
    
    if desired_timestep_minutes < 15:
        raise ValueError("get_load_profile is not set up for timesteps less than 15 minutes.")
    elif desired_timestep_minutes == 15:
        return timeseries * 4 # Convert from kWh to kW
    elif desired_timestep_minutes % 15 != 0: 
        raise ValueError("get_load_profile is not set up for that aren't evenly divisiable by 15. Pick 15, 30, or 60.")
    else:
        averaged_timesteps = []
        steps_per_step = desired_timestep_minutes / 15
        steps_per_hr = 60 / desired_timestep_minutes
        step = 0
        avg_kwhs = 0
        for kwh in timeseries:
             avg_kwhs += kwh
             step += 1
             if step == steps_per_step:
                 averaged_timesteps.append(avg_kwhs * steps_per_hr)
                 step = 0
                 avg_kwhs = 0
        return averaged_timesteps

In [4]:
def run_location(weather_file, rate_path, load_path):

    # Utility rate data is contained here since the battery needs it for dispatch calculations
    rate_setup = get_pysam_json(rate_path)
    wf_timestep = 60
    load_profile = get_load_profile(load_path, wf_timestep)

    pv = pv_model.default("PVBatteryResidential") # This runs both PV and battery
    ur = utility_rate.from_existing(pv, "PVBatteryResidential")
    cl = cashloan.from_existing(ur, "PVBatteryResidential")

    for k, v in rate_setup.items():
        try:
            pv.value(k, v)
        except AttributeError:
            if "batt_adjust" in k:
                pass
            else:
                print("Failed to assign PV key " + str(k))

    pv.value("solar_resource_file", str(weather_file))
    pv.value("batt_dispatch_choice", 4) # Retail rates dispatch
    pv.value("load", load_profile)

    pv.execute()
    ur.execute()
    cl.execute()

    output_data = {}
    output_data.update(pv.Outputs.export())
    output_data.update(ur.Outputs.export())
    output_data.update(cl.Outputs.export())
    return output_data


In [8]:
file_dir = os.path.abspath('') #ensure this is ""Users/bspeetle/Desktop/repo/SAM-analyses/2025/battwatts_sensitivity/"
weather_path = file_dir + "/weather_data/" + "AL" + "_" + "33.56" + "_" + "-86.75" + "_" + "33.56" + "_" + "-86.75" + "_nsrdb-GOES-aggregated-v4-0-0_60_2018.csv"
rate_path = file_dir + "/rate_data/" + "alabama" + "_" + "apc" + "_2025_pvsamv1.json"
load_path = file_dir + "/load_data/" + "load-data-" + "AL" + "-" + "46496" + "-0.csv"
test_output = run_location(weather_path,rate_path,load_path)
print("Keys in run_location output:")
for key in test_output.keys():
    print(key)


Keys in run_location output:
ac_gross
ac_lifetime_loss
ac_perf_adj_loss
ac_transmission_loss
ac_wiring_loss
airmass
alb
annual_ac_battery_loss_percent
annual_ac_gross
annual_ac_inv_clip_loss_percent
annual_ac_inv_eff_loss_percent
annual_ac_inv_pnt_loss_percent
annual_ac_inv_pso_loss_percent
annual_ac_lifetime_loss_percent
annual_ac_loss_ond
annual_ac_perf_adj_loss_percent
annual_ac_wiring_loss
annual_ac_wiring_loss_percent
annual_bifacial_electrical_mismatch
annual_bifacial_electrical_mismatch_percent
annual_dc_battery_loss_percent
annual_dc_diodes_loss
annual_dc_diodes_loss_percent
annual_dc_gross
annual_dc_inv_tdc_loss_percent
annual_dc_invmppt_loss
annual_dc_lifetime_loss_percent
annual_dc_loss_ond
annual_dc_mismatch_loss
annual_dc_mismatch_loss_percent
annual_dc_module_loss_percent
annual_dc_mppt_clip_loss_percent
annual_dc_nameplate_loss
annual_dc_nameplate_loss_percent
annual_dc_net
annual_dc_nominal
annual_dc_optimizer_loss
annual_dc_optimizer_loss_percent
annual_dc_perf_adj_los

To generate rate data:

- Open the SAM file in sam files
- Download the rate data on the utility rates page
- Make adjustments as needed (e.g. "net billing" for California rates)
- Use shift-F5 to export the code and choose "PySAM JSON"
- Save the untitled_pvsamv1.json file to the rate_data folder and rename it to something more descriptive

In [9]:
print(len(test_output))

526


In [10]:
file_dir = os.path.abspath('') #ensure this is ""Users/bspeetle/Desktop/repo/SAM-analyses/2025/battwatts_sensitivity/"

state_long = ["alabama","arizona","california","colorado","massachusetts","minnesota","newyork","ohio","oregon","texas","vermont","wisconsin"] #states in sensitivity analysis
state_short = ["AL","AZ","CA","CO","MA","MN","NY","OH","OR","TX","VT","WI"] #states in sensitivity analysis (abbreviated)
utility_names = ["apc","aps","sdge","xcel","nationalgrid","xcel","coned","aep","pge","xcel","gmp","we_energies"]
building_id = ["46496","112157","111549","126745","305","442412","404322","448494","67263","168160","1652","467842"]
latitude = ["33.56","35.14","32.87","39.81","41.88","44.85","40.68","39.99","45.59","35.22","44.47","42.95"]
longitude = ["-86.75","-111.67","-117.15","-105.14","-71.02","-93.03","-74.17","-82.88","-122.6","-101.71","-73.15","-87.9"]
all_results = []

for st_long, st_short, utility, bd_id, lat, lon in zip(
    state_long, state_short, utility_names, building_id, latitude, longitude
):
    rate_path = file_dir + "/rate_data/" + st_long + "_" + utility + "_2025_pvsamv1.json"
    weather_path_tmy = file_dir + "/weather_data/" + st_short + "_" + lat + "_" + lon + "_" + lat + "_" + lon + "_nsrdb-GOES-tmy-v4-0-0_60_tmy.csv"
    weather_path_amy = file_dir + "/weather_data/" + st_short + "_" + lat + "_" + lon + "_" + lat + "_" + lon + "_nsrdb-GOES-aggregated-v4-0-0_60_2018.csv"
    load_path = file_dir + "/load_data/" + "load-data-" + st_short + "-" + bd_id + "-0.csv"

    print(f"Running analysis for {st_short} ({st_long}), building ID {bd_id}...")

    tmy_outputs = run_location(weather_path_tmy, rate_path, load_path)
    amy_outputs = run_location(weather_path_amy, rate_path, load_path)

    annual_bills_tmy = tmy_outputs.get("utility_bill_w_sys", [None]*12)
    annual_bills_amy = amy_outputs.get("utility_bill_w_sys", [None]*12)

    # Create the filtered output for TMY
    filtered_tmy = {
        "savings_year1": tmy_outputs.get("savings_year1", 0),
        "annual_energy": tmy_outputs.get("annual_energy", 0),
        "npv": tmy_outputs.get("npv", 0),
        "year_1_bill": annual_bills_tmy[1]
    }

    # Create the filtered output for AMY (same logic)
    filtered_amy = {
        "savings_year1": amy_outputs.get("savings_year1"),
        "annual_energy": amy_outputs.get("annual_energy"),
        "npv": amy_outputs.get("npv"),
        "year_1_bill": annual_bills_amy[1]

    }



    result_row = {
        "state_long": st_long,
        "state_short": st_short,
        "utility": utility,
        "building_id": bd_id,
        "latitude": lat,
        "longitude": lon,
        "run_type": "tmy",
        **filtered_tmy  
    }
    all_results.append(result_row)

    result_row = {
        "state_long": st_long,
        "state_short": st_short,
        "utility": utility,
        "building_id": bd_id,
        "latitude": lat,
        "longitude": lon,
        "run_type": "amy",
        **filtered_amy
    }
    all_results.append(result_row)

Running analysis for AL (alabama), building ID 46496...
Running analysis for AZ (arizona), building ID 112157...
Running analysis for CA (california), building ID 111549...
Running analysis for CO (colorado), building ID 126745...
Running analysis for MA (massachusetts), building ID 305...
Running analysis for MN (minnesota), building ID 442412...
Running analysis for NY (newyork), building ID 404322...
Running analysis for OH (ohio), building ID 448494...
Running analysis for OR (oregon), building ID 67263...
Running analysis for TX (texas), building ID 168160...
Running analysis for VT (vermont), building ID 1652...
Running analysis for WI (wisconsin), building ID 467842...


In [12]:
output_file = "battwatts_sensitivity_outputs.csv"
keys = all_results[0].keys()  # use first result's keys as headers

with open(output_file, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(all_results)

print(f"Saved {len(all_results)} rows to {output_file}")
print(all_results)

#print(tmy_outputs["utility_bill_w_sys"][1])
#print(tmy_outputs["savings_year1"])
#print(tmy_outputs["annual_energy"])
#print(tmy_outputs["npv"])

#print(amy_outputs["utility_bill_w_sys"][1])
#print(amy_outputs["savings_year1"])
#print(amy_outputs["annual_energy"])
#print(amy_outputs["npv"])

#print("Energy difference " + str(amy_outputs["annual_energy"] / tmy_outputs["annual_energy"]) )
#print("Bill difference " + str(amy_outputs["savings_year1"] / tmy_outputs["savings_year1"]))




Saved 24 rows to battwatts_sensitivity_outputs.csv
[{'state_long': 'alabama', 'state_short': 'AL', 'utility': 'apc', 'building_id': '46496', 'latitude': '33.56', 'longitude': '-86.75', 'run_type': 'tmy', 'savings_year1': 996.5532829946998, 'annual_energy': 8861.195288683932, 'npv': -11327.57288126162, 'jan_bill': 0.0, 'feb_bill': 833.1051553737088, 'mar_bill': 862.3250673589891, 'apr_bill': 889.81903283468, 'may_bill': 918.7771533465518, 'jun_bill': 947.484378874185, 'jul_bill': 976.9910078854376, 'aug_bill': 1007.8273932276669, 'sep_bill': 1041.488952737007, 'oct_bill': 1074.0843859732317, 'nov_bill': 1106.2679384512703, 'dec_bill': 1140.1992458367515}, {'state_long': 'alabama', 'state_short': 'AL', 'utility': 'apc', 'building_id': '46496', 'latitude': '33.56', 'longitude': '-86.75', 'run_type': 'amy', 'savings_year1': 992.5525758074181, 'annual_energy': 8443.436586365393, 'npv': -11366.247726402078, 'jan_bill': 0.0, 'feb_bill': 837.1058625609895, 'mar_bill': 865.2665703334162, 'apr_b

In [ ]:
print(type(amy_outputs))
print(len(str(tmy_outputs)))